In [1]:
# current dir
%pwd

%cd '~/privacy_active_learning'

/Users/andreasopsahlferstad/privacy_active_learning


In [2]:
from xgboost.sklearn import XGBClassifier


In [4]:
from input.voice.voice_import import get_voice_data

In [5]:
data = get_voice_data()

variance kept due to PCA: 0.97


In [6]:
train = data.sample(n=500)

In [7]:
X = train.iloc[:,1:]
y = train.iloc[:,0]

In [8]:
xg = XGBClassifier(learning_rate=0.1,
            n_estimators=1000,
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='multi:softmax',
            nthread=8,
            scale_pos_weight=1,
            seed=0)

In [18]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2436,6.411785,9.227964,5.085998,-2.368798,5.719873,-2.903780,-6.164981,-1.407622,-9.371026,1.135372,...,0.039405,0.001824,-0.539755,0.352398,0.022815,0.313504,0.673618,-0.320396,0.432380,-0.076211
1519,10.117870,-10.185149,-4.645674,-1.961590,-2.704262,3.472707,1.548210,-2.004890,0.958254,4.558173,...,0.112705,-0.051735,-0.174975,-0.160404,-0.140225,0.032943,-0.305237,0.454419,0.006048,0.120737
4185,-11.653149,5.854071,14.963570,8.374763,0.893180,-0.655721,6.707217,10.350585,6.247142,3.184425,...,-0.108723,-0.130477,0.200257,0.016359,-0.196137,-0.191710,-0.574974,0.114481,0.206067,-0.021371
4623,-12.479951,0.718186,5.778529,-1.094971,6.891340,0.728841,-4.024922,2.149929,-2.552387,0.471854,...,0.422592,0.066267,-0.133395,-0.472080,-0.011764,0.108096,-0.937192,0.761661,0.156557,-0.139952
4029,-19.117382,3.280064,10.835315,10.628334,-3.448583,2.153458,3.554593,1.354874,-3.870609,5.259204,...,0.219557,0.440156,0.353140,0.282902,0.495037,0.051441,0.288239,0.102125,-0.108209,0.380697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6968,-13.252746,-1.799584,-10.791623,-0.478957,3.080260,0.461633,-0.411095,0.923135,-4.357333,2.617894,...,0.067390,-0.308404,0.219051,0.228290,0.228868,-0.463160,-0.220504,-0.096232,0.195210,-0.386038
3454,-7.245454,9.533084,2.975926,1.094939,-3.450215,5.026801,4.080831,-0.620966,1.179843,10.649586,...,0.709527,1.073483,0.180284,-0.805631,-0.762835,0.218174,-1.607572,0.048808,0.643282,0.338077
2671,2.210760,1.670858,13.583397,1.088343,-2.329607,-1.587633,-2.108068,-3.651491,-1.332405,-4.876875,...,0.446595,0.165958,0.204113,0.542397,0.433225,-0.097307,0.618107,0.020236,-0.264366,-0.411139
6845,20.676349,-3.878379,5.584947,-0.328947,0.016768,10.027784,-0.069605,3.790430,0.126391,-4.242539,...,-0.118747,-0.130776,-0.413523,0.265089,0.084302,0.149528,0.105920,-0.276486,-0.181587,0.236658


In [9]:
xg.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=8, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=None, subsample=0.8, verbosity=1)

In [11]:
test = data.drop(X.index)
X_test = test.iloc[:,1:]
y_test = test.iloc[:,0]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-9.419547,0.459947,-7.288776,4.110508,-6.179064,-1.794985,-0.001060,3.682690,2.262369,-0.754340,...,0.426717,-0.391139,-0.190100,-0.044736,-0.246842,-0.032605,0.151998,-0.047171,-0.054338,0.252969
1,-5.124728,-1.297131,-7.837942,2.811276,-2.980114,-3.179390,-3.616721,-1.658861,0.993507,1.924557,...,0.476081,0.707167,0.027435,-0.201637,0.974762,-0.146275,0.102927,0.754423,0.149169,0.175828


In [22]:
import numpy as np
np.sum(xg.predict_proba(X_test.iloc[:2,:]), axis = 1)

array([0.99999994, 0.99999994], dtype=float32)

In [23]:
from scipy.stats import entropy


In [26]:
entropy([0.5, 0.5], base = 2)

1.0